In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
tweets = pd.read_csv('tweets.csv')

In [3]:
tweets = tweets[~tweets.user_id.isna()]

tweets.loc[:,'isRT'] = tweets.text.apply(lambda x: False if pd.isna(x) else True if re.match(r'^RT @',x) else False)

In [4]:
tweets[(tweets.isRT==True) & (~tweets.bot)].sample(300)

,Unnamed: 0,id,text,user_id,in_reply_to_user_id,retweeted_status_id,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,created_at,timestamp,crawled_at,updated,Cat,bot,isRT
4096107,4096107,563107121341136896,RT @JasperDolphin: Yeah I need some top,1.882245e+09,True,True,203.0,0.0,0.0,0.0,0.0,1.0,Wed Feb 04 22:49:34 +0000 2015,2015-02-04 23:49:34,2015-05-01 13:22:58,2015-05-01 13:22:58,Humans,False,True
4506148,4506148,593717423938871296,RT @BillyZane: #DoubleThrowbackThursday Twin P...,5.348635e+08,True,True,18.0,0.0,0.0,1.0,0.0,1.0,Thu Apr 30 10:03:59 +0000 2015,2015-04-30 12:03:59,2015-05-01 13:37:04,2015-05-01 13:37:04,Humans,False,True
4238479,4238479,523542116199563264,RT @kbris25: Never let em know what your next ...,1.911525e+09,True,True,3.0,0.0,0.0,0.0,0.0,0.0,Sat Oct 18 18:32:22 +0000 2014,2014-10-18 20:32:22,2015-05-01 13:28:06,2015-05-01 13:28:06,Humans,False,True
3487506,3487506,547977549813673985,RT @officialDannyT: Merry Christmas and Happy ...,1.813221e+06,True,True,3301.0,0.0,0.0,0.0,0.0,1.0,Thu Dec 25 04:50:03 +0000 2014,2014-12-25 05:50:03,2015-05-01 13:00:54,2015-05-01 13:00:54,Humans,False,True
4922537,4922537,585809988662099968,RT @Tee5588: @MarkKriski right on Point! Now r...,1.845981e+07,True,True,1.0,0.0,0.0,0.0,0.0,2.0,Wed Apr 08 14:22:39 +0000 2015,2015-04-08 16:22:39,2015-05-01 13:54:24,2015-05-01 13:54:24,Humans,False,True
6091202,6091202,589860041307594752,RT @luhanpics: 1 hr and 40 mins more! (CST) #H...,2.325932e+09,True,True,28.0,0.0,0.0,1.0,0.0,1.0,Sun Apr 19 18:36:07 +0000 2015,2015-04-19 20:36:07,2015-05-01 16:06:06,2015-05-01 16:06:06,Humans,False,True
4014019,4014019,562042577956773889,"RT @xopurified: If you're going to cheat, why ...",1.858934e+09,True,True,396.0,0.0,0.0,0.0,0.0,1.0,Mon Feb 02 00:19:27 +0000 2015,2015-02-02 01:19:27,2015-05-01 13:20:15,2015-05-01 13:20:15,Humans,False,True
4111891,4111891,590903182412283904,RT @GyanendrAw: https://t.co/OIi6quNgIu Kumar ...,1.496966e+07,True,True,3.0,0.0,0.0,0.0,1.0,1.0,Wed Apr 22 15:41:11 +0000 2015,2015-04-22 17:41:11,2015-05-01 13:23:27,2015-05-01 13:23:27,Humans,False,True
5932399,5932399,520015830541533184,RT @GenericEntity: So my best friend @nightwin...,2.307118e+09,True,True,1.0,0.0,0.0,0.0,0.0,2.0,Thu Oct 09 01:00:10 +0000 2014,2014-10-09 03:00:10,2015-05-01 15:59:44,2015-05-01 15:59:44,Humans,False,True
5583977,5583977,515956573726715905,RT @HeyItsLaurenox: The Walking Dead returns i...,2.157773e+08,True,True,7.0,0.0,0.0,0.0,0.0,1.0,Sat Sep 27 20:10:07 +0000 2014,2014-09-27 22:10:07,2015-05-01 15:28:13,2015-05-01 15:28:13,Humans,False,True


In [5]:
tweets.loc[3960355].user_id

14443013.0

In [6]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6296494 entries, 0 to 6296493
Data columns (total 19 columns):
Unnamed: 0             int64
id                     int64
text                   object
user_id                float64
in_reply_to_user_id    bool
retweeted_status_id    bool
retweet_count          float64
reply_count            float64
favorite_count         float64
num_hashtags           float64
num_urls               float64
num_mentions           float64
created_at             object
timestamp              object
crawled_at             object
updated                object
Cat                    object
bot                    bool
isRT                   bool
dtypes: bool(4), float64(7), int64(2), object(6)
memory usage: 792.6+ MB


In [7]:
len(tweets), 'tweets'

(6296494, 'tweets')

In [8]:
#tweets_humans = tweets[tweets.Cat=='Humans']
#display(tweets_humans.columns)

#first = True
def getTimes(times):
    if(len(times) == 0):
        return np.array([0,0,0,0])
    return np.array([times.mean().seconds, times.median().seconds, times.std().seconds, times.quantile(0.25).seconds, times.quantile(0.75).seconds])

def getDist(var):
    if(len(var) == 0):
        return np.array([0,0,0,0])
    return np.array([var.mean(), var.median(), var.std(), var.quantile(0.25), var.quantile(0.75)])

def getWeekAv(tw):
    return len(tw) / len(tw.weekNum.unique())

def getDataTweets(twU):
    twU = twU.iloc[0:300]
    twU = twU.sort_values(by='timestamp', ascending=0).fillna(0) #tweets del usuario
    dates = pd.to_datetime(twU.timestamp) #aca lo que hago es guardar la columnoa como datetime

    twU.loc[:,'dayOfWeek'] = dates.dt.dayofweek # dia de la semana
    twU.loc[:,'hour'] = dates.dt.hour #hora del tweet
    twU.loc[:,'weekNum'] = dates.apply(lambda x: str(x.week) + '-' + str(x.year))

    tweetsUser = twU[twU.isRT==False] #tweets propios del usuario
    tweetsRT = twU[twU.isRT==True] #retweets

    twU.loc[:,'interval'] = pd.to_datetime(twU.timestamp.shift(1)) - pd.to_datetime(twU.timestamp)
    times = pd.to_datetime(twU.timestamp.shift(1)) - pd.to_datetime(twU.timestamp) # calculo distancia entre tweets

    tweetsUser.loc[:,'interval'] = pd.to_datetime(tweetsUser.timestamp.shift(1)) - pd.to_datetime(tweetsUser.timestamp)
    timesUser = pd.to_datetime(tweetsUser.timestamp.shift(1)) - pd.to_datetime(tweetsUser.timestamp) # calculo distancia entre tweets

    tweetsRT.loc[:,'interval'] = pd.to_datetime(tweetsRT.timestamp.shift(1)) - pd.to_datetime(tweetsRT.timestamp)
    timesRT = pd.to_datetime(tweetsRT.timestamp.shift(1)) - pd.to_datetime(tweetsRT.timestamp) # calculo distancia entre tweets
    
    mediaSemanas = np.around(twU.groupby('dayOfWeek').apply(getWeekAv), 6)
    
    #ver word_to_vect
    result = np.concatenate([
        [
            twU.iloc[0].user_id,
            twU.iloc[0].bot,
            twU.iloc[0].Cat,
            twU.id.count(),
            len(twU.weekNum.unique())
        ],
        mediaSemanas,
        getTimes(times),
        getTimes(timesRT),
        getTimes(timesUser),
        getDist(tweetsUser.favorite_count) * 10000,
        getDist(tweetsUser.retweet_count) * 10000,
        getDist(tweetsUser.num_mentions) * 10000,
        getDist(tweetsUser.reply_count) * 10000,

    ])
    
    return result
#         display(twU.head())
    #time_median, time_std, max_cuart, min_cuart, media_lun, media_martes, dispercion_horario
    #media_palabras
    #cantidad de palabras unicas

    #cantidad_usr que retwittea / cantidad de retweets

# res = pd.concat([
#     tweets[tweets.bot==True].iloc[0:6000],
#     tweets[tweets.bot==False].iloc[0:6000]
# ]).groupby('user_id').apply(getDataTweets)

res = tweets.groupby('user_id').apply(getDataTweets)
cols = np.concatenate(
    [
        
        [x+'_mean',x+'_median', x+'_std', x+'_fq', x+'_tq'] for x in
        ['times','timesRT','timesUser','favorite_count','retweet_count','num_mentions','reply_count']
    ]
)
cols = np.concatenate([
    ['user_id', 'bot', 'Cat', 'tweetsCount', 'semanas', 'lun', 'mar', 'mie', 'jue', 'vie', 'sab', 'dom'],cols
])
res = pd.DataFrame(
    [x for x in res], columns = cols
)
res.index = res.user_id

print(len(res))

5995


In [9]:
display(res.head())
print(res.columns)

,user_id,bot,Cat,tweetsCount,semanas,lun,mar,mie,jue,vie,...,num_mentions_mean,num_mentions_median,num_mentions_std,num_mentions_fq,num_mentions_tq,reply_count_mean,reply_count_median,reply_count_std,reply_count_fq,reply_count_tq
user_id,,,,,,,,,,,,,,,,,,,,,
678033.0,678033.0,False,Humans,300,4,12.666667,17.666667,8.333333,19.333333,10.0,...,9313.304721030043,10000.0,8781.379387404811,0.0,10000.0,0.0,0.0,0.0,0.0,0.0
722623.0,722623.0,False,Humans,300,10,4.444444,4.0,3.888889,4.6,4.777778,...,5591.836734693878,0.0,6086.528955598055,0.0,10000.0,0.0,0.0,0.0,0.0,0.0
755116.0,755116.0,False,Humans,300,5,4.666667,21.0,7.0,28.0,8.8,...,7824.561403508772,10000.0,8609.53207210577,0.0,10000.0,0.0,0.0,0.0,0.0,0.0
755746.0,755746.0,False,Humans,300,10,4.333333,3.9,4.4,5.875,6.0,...,6236.162361623617,10000.0,6073.89240287389,0.0,10000.0,0.0,0.0,0.0,0.0,0.0
785080.0,785080.0,False,Humans,300,3,14.333333,36.0,27.333333,3.333333,22.0,...,6996.336996336996,0.0,9142.860509587823,0.0,10000.0,0.0,0.0,0.0,0.0,0.0


Index(['user_id', 'bot', 'Cat', 'tweetsCount', 'semanas', 'lun', 'mar', 'mie',
       'jue', 'vie', 'sab', 'dom', 'times_mean', 'times_median', 'times_std',
       'times_fq', 'times_tq', 'timesRT_mean', 'timesRT_median', 'timesRT_std',
       'timesRT_fq', 'timesRT_tq', 'timesUser_mean', 'timesUser_median',
       'timesUser_std', 'timesUser_fq', 'timesUser_tq', 'favorite_count_mean',
       'favorite_count_median', 'favorite_count_std', 'favorite_count_fq',
       'favorite_count_tq', 'retweet_count_mean', 'retweet_count_median',
       'retweet_count_std', 'retweet_count_fq', 'retweet_count_tq',
       'num_mentions_mean', 'num_mentions_median', 'num_mentions_std',
       'num_mentions_fq', 'num_mentions_tq', 'reply_count_mean',
       'reply_count_median', 'reply_count_std', 'reply_count_fq',
       'reply_count_tq'],
      dtype='object')


In [ ]:
# sns.pairplot(res[['times_mean','timesRT_mean','Cat']], hue='Cat', diag_kind="kde")


g = sns.scatterplot(x="times_mean", y="timesRT_mean", hue='Cat', data=res, alpha=0.5, size=2)

g.set(xlim=(0, 15000))

[(0, 15000)]

In [13]:

import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
plotly.__version__

init_notebook_mode(connected=True)

def plotearVars(variables):
    colors = {'Humans':'red', 'SocialSpambot1':'blue', 'SocialSpambot2':'green', 'SocialSpambot3':'black'}
    trace1 = go.Scatter3d(
        x=res[variables[0][0]],
        y=res[variables[1][0]],
        z=res[variables[2][0]],
        mode='markers',
        marker=dict(
            size=4,
            color=res['Cat'].apply(lambda x: colors[x]),
            line=dict(
                color='rgba(255,255,255,0.2)',
                width=0
            ),
            opacity=0.3
        )
    )
    layout = go.Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        ),
        scene = dict(
            xaxis = dict(
                nticks=4,
                range = (lambda: None if len(variables[0])==1 else variables[0][1])(),
                title=variables[0][0],
            ),
            yaxis = dict(
                range = (lambda: None if len(variables[1])==1 else variables[1][1])(),
                title=variables[1][0],
            ),
            zaxis = dict(
                range = (lambda: None if len(variables[2])==1 else variables[2][1])(),
                title= variables[2][0],
            ),
        )
    )
    print(  )
    fig = go.Figure(data=[trace1], layout=layout)
    iplot(fig)
    
plotearVars([
    ['times_median', [0,10000]],
    ['lun', [0,100]],
    ['timesUser_tq', [0,100000]],
])

In [15]:
plotearVars([
    ['times_tq'],
    ['favorite_count_mean', [0, 60000]],
    ['timesUser_tq', [0,100000]],
])

In [16]:
res.to_csv('features_generadas.csv')